<a href="https://colab.research.google.com/github/PavlenkoDR/AbstractFab/blob/master/Neuro_Lizka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Распознавание цветов (ромашки, розы, подсолнухи, тюльпаны, одуванчики) на изображениях с помощью предварительно обученной нейронной сети VGG16

Для распознавания используется предварительно обученная сверточная нейронная сеть VGG16.

Перед использованием необходимо скачать и подготовить данные для обучения, проверки и тестирования.

In [0]:
import numpy
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.applications import VGG16
from tensorflow.python.keras.optimizers import Adam

from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.python.keras.layers.normalization import BatchNormalization

from tensorflow.python.keras.applications import InceptionV3


from keras.datasets import cifar10
from keras.utils import np_utils
from keras.optimizers import SGD

In [0]:
# Задаем seed для повторяемости результатов
numpy.random.seed(42)

# Загружаем данные
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
# Размер мини-выборки
batch_size = 32
# Количество классов изображений
nb_classes = 10
# Количество эпох для обучения
nb_epoch = 25
# Размер изображений
img_rows, img_cols = 32, 32
# Количество каналов в изображении: RGB
img_channels = 3

# Нормализуем данные
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# Преобразуем метки в категории
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

Загружаем предварительно обученную нейронную сеть

In [0]:
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
# Загружаем предварительно обученную нейронную сеть Inceptionv3
#inception_net = InceptionV3(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

Создаем составную нейронную сеть на основе VGG16 или inception

In [0]:
vgg16_net.summary()
#inception_net.summary()

In [0]:
model = Sequential()
# Добавляем в модель сеть VGG16 вместо слоя
model.add(vgg16_net)
#model.add(inception_net)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
#model.add(Dense(512, activation='sigmoid'))
model.add(Dense(nb_classes, activation='softmax'))

Компилируем составную нейронную сеть

In [0]:
# Задаем параметры оптимизации
#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-6),
              metrics=['accuracy'])

Создаем генератор изображений
Генератор изображений создается на основе класса ImageDataGenerator. Генератор делит значения всех пикселов изображения на 255. - ВОТ ЭТО СКОРЕЕ ВСЕГО НЕ НУЖНО


In [0]:
datagen = ImageDataGenerator(rescale=1./255)

In [92]:

model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=nb_epoch,
              validation_split=0.1,
              shuffle=True,
              verbose=2)

Train on 45000 samples, validate on 5000 samples
Epoch 1/25
45000/45000 - 56s - loss: 1.9842 - acc: 0.2951 - val_loss: 1.3243 - val_acc: 0.5774
Epoch 2/25
45000/45000 - 55s - loss: 1.3716 - acc: 0.5304 - val_loss: 1.0446 - val_acc: 0.6532
Epoch 3/25
45000/45000 - 55s - loss: 1.1725 - acc: 0.6017 - val_loss: 0.9326 - val_acc: 0.6816
Epoch 4/25
45000/45000 - 55s - loss: 1.0643 - acc: 0.6415 - val_loss: 0.8597 - val_acc: 0.7018
Epoch 5/25
45000/45000 - 55s - loss: 0.9896 - acc: 0.6647 - val_loss: 0.8142 - val_acc: 0.7150
Epoch 6/25
45000/45000 - 55s - loss: 0.9306 - acc: 0.6851 - val_loss: 0.7820 - val_acc: 0.7274
Epoch 7/25
45000/45000 - 55s - loss: 0.8821 - acc: 0.7048 - val_loss: 0.7468 - val_acc: 0.7380
Epoch 8/25
45000/45000 - 53s - loss: 0.8498 - acc: 0.7153 - val_loss: 0.7281 - val_acc: 0.7414
Epoch 9/25
45000/45000 - 55s - loss: 0.8180 - acc: 0.7244 - val_loss: 0.7125 - val_acc: 0.7512
Epoch 10/25
45000/45000 - 55s - loss: 0.7820 - acc: 0.7383 - val_loss: 0.6926 - val_acc: 0.7548


In [95]:
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))

Точность работы на тестовых данных: 79.59%
